In [1]:
from pokemontcgsdk import Card
from pokemontcgsdk import Set
from pokemontcgsdk import Type
from pokemontcgsdk import Supertype
from pokemontcgsdk import Subtype
from pokemontcgsdk import Rarity

from pokemontcgsdk import RestClient

with open("options.txt", "r") as file:
    option_lines = file.readlines()
    
api_key = option_lines[0].split(" = ")[1]

RestClient.configure(api_key)

In [2]:
sets = Set.all()

In [4]:
for card_set in sets:
    print(card_set)

Set(id='base1', images=SetImage(symbol='https://images.pokemontcg.io/base1/symbol.png', logo='https://images.pokemontcg.io/base1/logo.png'), legalities=Legality(unlimited='Legal', expanded=None, standard=None), name='Base', printedTotal=102, ptcgoCode='BS', releaseDate='1999/01/09', series='Base', total=102, updatedAt='2022/10/10 15:12:00')
Set(id='base2', images=SetImage(symbol='https://images.pokemontcg.io/base2/symbol.png', logo='https://images.pokemontcg.io/base2/logo.png'), legalities=Legality(unlimited='Legal', expanded=None, standard=None), name='Jungle', printedTotal=64, ptcgoCode='JU', releaseDate='1999/06/16', series='Base', total=64, updatedAt='2020/08/14 09:35:00')
Set(id='basep', images=SetImage(symbol='https://images.pokemontcg.io/basep/symbol.png', logo='https://images.pokemontcg.io/basep/logo.png'), legalities=Legality(unlimited='Legal', expanded=None, standard=None), name='Wizards Black Star Promos', printedTotal=53, ptcgoCode='PR', releaseDate='1999/07/01', series='Ba